Trong notebook này, bằng cách sử dụng Transformer, chúng ta thực hiện bài toán như một **question-answring system**.

Code và notebook được thiết kế rõ ràng để hiểu cho người mới bắt đầu nhưng hy vọng cũng có ích cho các Kaggler nâng cao.

Bất kỳ bình luận / phản hồi là rất đánh giá cao.  Tuyên bố từ chối trách nhiệm: công việc đang được tiến hành, tôi sẽ sớm thêm các tài nguyên và nhận xét mới.

# 1. Problem formulation
Chúng tôi xây dựng bài toán question answring: cho một câu hỏi và mộ ngữ cảnh, chúng tôi huấn luyện transformer để tìm **answer** trong cột **text** (the context).

Chúng tôi có:
1. Question: **sentiment** column (**positive** hoặc **negative**)
2. Context: **text** column
3. Answer: **selected_text** column

# 2. Getting started with QA
# 3. Learning QA from scratch
# 4. Model: DistilBERT + SquAD
# 5. Dataset publicly avaiable


## 5.1 DistilBERT + SQuAD model
Bởi vì sổ ghi chép của Tweet Sentiment Extraction phải tắt internet, tôi đã tải xuống và lưu trữ mô hình máy biến áp trong tập dữ liệu Kaggle công khai: Các mô hình distilBERT được đào tạo trước của Transformers.  Trong tương lai, tôi dự định tải tất cả các mô hình đã được đào tạo trước của distilBERT lên cùng một tập dữ liệu để chúng tôi có thể dễ dàng thử nghiệm với nhiều mô hình và cấu hình.

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sub_df = pd.read_csv('./data/sample_submission.csv')

In [3]:
train = np.array(train_df)
test = np.array(test_df)

In [4]:
use_cuda = True

In [5]:
train_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


```
train_data = [
    {
        'context': "This tweet sentiment extraction challenge is great",
        'qas': [
            {
                'id': "00001",
                'question': "positive",
                'answers': [
                    {
                        'text': "is great",
                        'answer_start': 43
                    }
                ]
            }
        ]
    }
    ]
```

In [7]:
%%time

""" Pre training data in QA-compatible format"""
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i==-1:
            return l1
        l1.append(i)
        index = i+1
    return l1

def do_qa_train(train):
    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]

        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue

        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'im_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})

    return output

qa_train = do_qa_train(train)

with open('./data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

nan <class 'float'>
nan <class 'float'>
neutral <class 'str'>
CPU times: user 735 ms, sys: 18.4 ms, total: 754 ms
Wall time: 852 ms


In [8]:
%%time

"""
Prepare testing data in QA-compatible format
"""

def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile)

CPU times: user 200 ms, sys: 3.55 ms, total: 204 ms
Wall time: 217 ms


In [9]:
from simpletransformers.question_answering import QuestionAnsweringModel
MODEL_PATH = './distilbert-base-uncased-distilled-squad/'

model = QuestionAnsweringModel('distilbert',
MODEL_PATH,
args = {'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    },
                              use_cuda=use_cuda
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at ./distilbert-base-uncased-distilled-squad/ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.train_model('data/train.json')

convert squad examples to features:   0%|          | 0/27480 [00:00<?, ?it/s]/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_l

/home/thang/env/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/home/thang/env/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



(10305, 0.7631413049680036)

In [22]:
%%time

predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions[0])

sub_df['selected_text'] = predictions_df['answer']

sub_df.to_csv('submission.csv', index=False)

print("File submitted successfully.")

convert squad examples to features:   0%|          | 0/3534 [00:00<?, ?it/s]/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/home/thang/env/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_le


File submitted successfully.
CPU times: user 1min 6s, sys: 1.01 s, total: 1min 7s
Wall time: 2min 40s
